# Data Collection for Defi Data:  

###### Importing transactional data using etherscan api

In [ ]:
# For Blockchain Data of an account
#importing libraries

import pandas as pd
import numpy as np
from requests import get
from matplotlib import pyplot as plt
from datetime import datetime

#Defining variables

API_KEY = "INSERT YOUR API KEY HERE"
address = "INSERT YOUR ADDRESS HERE"
BASE_URL = "https://api.etherscan.io/api"
ETHER_VALUE = 10 ** 18

#Importing the balance of an account from ethereum blockchain via API KEY and Address

def make_api_url(module, action, address, **kwargs):
    url = BASE_URL + f"?module={module}&action={action}&address={address}&apikey={API_KEY}"

    for key, value in kwargs.items():
        url += f"&{key}={value}"

    return url 

def get_account_balance(address):
    balance_url = make_api_url("account", "balance", address, tag="latest")
    response = get(balance_url)
    data = response.json()

    value = int(data["result"]) / ETHER_VALUE
    return value
eth = get_account_balance(address)
print(eth)

In [ ]:
#Importing the transactions of an account from ethereum blockchain via API KEY and Address

def get_transactions(address):
    global data
    global data2
    global data3
    
#Normal "Native" Transactions
    transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]
    
#Internal "Native" Transactions

    internal_tx_url = make_api_url("account", "txlistinternal", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response2 = get(internal_tx_url)
    data2 = response2.json()["result"]

    data.extend(data2)
    
#ERC20 "Token" Transactions 

    erc20_transactions_url = make_api_url("account", "tokentx", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(erc20_transactions_url)
    data3 = response.json()["result"]
    data.extend(data3)
    data.sort(key=lambda x: int(x['timeStamp']))
    print(data)
    
get_transactions(address)


In [ ]:
#Converting transactions dictionary into dataframe

import csv

output_data = data

with open('output.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(output_data[0].keys())  # Write header row

    for row in output_data:
        writer.writerow(row.values())

file.close()

df = pd.DataFrame(output_data)

df.to_csv('output.csv', index=False)

In [ ]:
pd.set_option('display.max_columns',None)
df = pd.read_csv('F:\output.csv')
df

###### Cleaning/ Organizing data

In [ ]:
#Converting TimeStamp column into string
df["timeStamp"] = df["timeStamp"].astype(str)

#Converting TimeStamp column from unix format to more simple format
df["timeStamp"] = pd.to_datetime(df["timeStamp"],unit = "s")

df = df.sort_values(by='timeStamp')

In [ ]:
#Creating a new column "Buy" based on following condition
conditions = [
    (df['to'] == 'INSERT YOUR ADDRESS HERE')
]
results = [df['value']]
df['Buy'] = np.select(conditions, results)

In [ ]:
#Creating a new column "Sell" based on following condition
conditions = [
    (df['from'] == 'INSERT YOUR ADDRESS HERE')
]
results = [df['value']]
df['Sell'] = np.select(conditions, results)

In [ ]:
#Creating a new column "Buy Cur." based on following condition
conditions = [
    (df['to'] == 'INSERT YOUR ADDRESS HERE')
]
results = [df['tokenSymbol']]
df['Buy Cur.'] = np.select(conditions, results)

In [ ]:
#Creating a new column "Sell Cur." based on following conditi
conditions = [
    (df['from'] == 'INSERT YOUR ADDRESS HERE')
]
results = [df['tokenSymbol']]
df['Sell Cur.'] = np.select(conditions, results)

In [ ]:
#Filling null values with 0
df['Buy Cur.'] = df['Buy Cur.'].fillna(0)
df['Sell Cur.'] = df['Sell Cur.'].fillna(0)

In [ ]:
#Replacing 0 with "ETH"
df['Buy Cur.'] = df['Buy Cur.'].replace(0,'ETH')
df['Sell Cur.'] = df['Sell Cur.'].replace(0,'ETH')

In [ ]:
# Calculating fee amount by multiplying Gas Used with Gas price and diving it by ether value
df['Fee'] =  (df["gasUsed"]) * (df["gasPrice"]) / ETHER_VALUE
#Defining fee currency
df['Fee Cur.'] = "ETH"

In [ ]:
#Creating a column of "Type" based of following condition:
conditions = [
    (df['Buy'] == df['Sell']),
    (df['from'] == 'INSERT YOUR ADDRESS HERE'),
    (df['to'] == 'INSERT YOUR ADDRESS HERE')
]
results = ["Other Fees","Deposit","Withdrawal"]
df['Type'] = np.select(conditions, results)

In [ ]:
df.rename(columns={"timeStamp": "Date"}, inplace=True)

In [ ]:
df['Exchange'] = "ETH wallet"

In [ ]:
# deleing extra columns
df = df.drop(["blockNumber", "hash", "nonce", "blockHash", "transactionIndex", "value", "gas", "gasPrice", "isError", "txreceipt_status", "input","contractAddress","cumulativeGasUsed", "gasUsed" ,"confirmations", "methodId", "functionName", "tokenName", "tokenSymbol", "tokenDecimal"],axis=1)

In [ ]:
#Reindexing
df = df.reindex(columns=["Type","Buy","Buy Cur.","Sell","Sell Cur.","Fee","Fee Cur.","to","from","Date"])

# Importing Cefi Data

In [ ]:
#Importing CEFI Data
df_cefi = pd.read_excel(r'F:\act_cefidata_11.xlsx')

# Merging Defi and Cefi Datasets into one dataset:

In [ ]:
#Combining both data sets
BIGDATA = pd.concat([df, df_cefi])
BIGDATA

### Cleaning and organizing combines dataset

In [ ]:
#Dropping useless columns
df = df.drop(["Group", "Comment"],axis=1)

In [ ]:
#Checking data types
df.dtypes

In [ ]:
#Converting "Date" column to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = pd.to_datetime(df['Date']).dt.normalize()

In [ ]:
#Sorting values
df = df.sort_values('Date')
df

In [ ]:
#Filtering Data
# Define the list of values you want to keep in the "Buy Cur." and "Sell Cur." columns
desired_values = ['ETH', 'USDT', 'BTC', 'OMG', 'USD', 'USDC', 'ENG', 'CRV', 'REN', 'USDC', 'XZC',
                  'YFI', 'RDN', 'BNB', 'XLM', 'GTO', 'CND', 'XMR', 'ICX', 'EOS', 'MKR', 'WPR']

# Create boolean masks for rows with valid values in "Buy Cur." or "Sell Cur." columns
valid_buy_cur = df['Buy Cur.'].isin(desired_values)
valid_sell_cur = df['Sell Cur.'].isin(desired_values)

# Filter the DataFrame to keep rows with desired currencies in both columns or if one column is NaN
df2 = df[(valid_buy_cur & valid_sell_cur) | (valid_buy_cur & df['Sell Cur.'].isna()) | (valid_sell_cur & df['Buy Cur.'].isna())]

# Reset the index, but keep the existing index as a new column
df2 = df2.reset_index(drop=False)

# Print the filtered dataset
df2

In [ ]:
#Checking unique values in columns:
for column in df2.columns:
    unique_values = df2[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print("\n")

# Accounting

### Importing OHLCV Data for each currency for the given time period using Binance api

In [ ]:
import ccxt
import pandas as pd

# Create an instance of the Binance exchange
exchange = ccxt.binance()
exchange.timeout = 30000
# Define the symbols for the cryptocurrencies
symbols = ['CRV/USDT', 'XMR/USDT', 'NEO/USDT', 'BTC/USDT', 'BNB/USDT', 'MTL/USDT', 'ZRX/USDT', 'YFI/USDT', 'ICX/USDT', 'EOS/USDT', 'AION/USDT', 'XLM/USDT', 'OMG/USDT', 'XZC/USDT', 'ETH/USDT', 'REN/USDT', 'LTC/USDT', 'HOT/USDT', 'MKR/USDT', 'GTO/USDT']

# Define the date range
start_date = pd.to_datetime('15 January, 2018')
end_date = pd.to_datetime('4 December, 2023')

# Define the interval for fetching data (1 day)
interval = 86400 * 1000  # 1 day in milliseconds

# Fetch the historical rates for each symbol
dfs = []
for symbol in symbols:
    all_rates = []
    current_date = start_date
    while current_date <= end_date:
        start_timestamp = int(current_date.timestamp() * 1000)
        end_timestamp = int((current_date + pd.DateOffset(days=1)).timestamp() * 1000)
        rates = exchange.fetch_ohlcv(symbol, timeframe='1d', since=start_timestamp, limit=1000)
        all_rates.extend(rates)
        current_date += pd.DateOffset(days=1)

    symbol_df = pd.DataFrame(all_rates, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    symbol_df['date'] = pd.to_datetime(symbol_df['timestamp'], unit='ms')
    symbol_df.set_index('date', inplace=True)
    symbol_df.drop(columns=['timestamp'], inplace=True)
    symbol_df.columns = [symbol.replace('/', '_') + '_' + column.lower() for column in symbol_df.columns]

    # Drop duplicates based on the 'date' column
    symbol_df = symbol_df[~symbol_df.index.duplicated(keep='first')]

    dfs.append(symbol_df)

# Merge the individual DataFrames into a single DataFrame
merged_df = pd.concat(dfs, axis=1)

# Print the merged DataFrame
print(merged_df)


### Importing today's rate for each currency 

In [ ]:
import ccxt
import datetime

# Initialize the Binance exchange instance
exchange = ccxt.binance({
    'rateLimit': 1200,  # Adjust the rate limit as needed
    'enableRateLimit': True,
})

# Load the Binance markets
markets = exchange.load_markets()

# List of trading pairs
symbols = ['CRV/USDT', 'XMR/USDT', 'NEO/USDT', 'BTC/USDT', 'BNB/USDT', 'MTL/USDT', 'ZRX/USDT', 'YFI/USDT', 'ICX/USDT', 'EOS/USDT', 'AION/USDT', 'XLM/USDT', 'OMG/USDT', 'XZC/USDT', 'ETH/USDT', 'REN/USDT', 'LTC/USDT', 'HOT/USDT', 'MKR/USDT', 'GTO/USDT']

# Specify the date for which you want to retrieve the closing prices
target_date = datetime.datetime(2023, 8, 7)

closing_prices = {}

for symbol in symbols:
    if symbol in markets:
        ccxt_symbol = markets[symbol]['symbol']
        
        # Fetch historical OHLCV data
        ohlcv = exchange.fetch_ohlcv(ccxt_symbol, '1d', since=int(target_date.timestamp()) * 1000)
        
        if len(ohlcv) > 0:
            closing_price = ohlcv[0][4]  # Closing price is at index 4
            closing_prices[symbol] = closing_price
    else:
        print(f"Symbol {symbol} not found in markets.")

# Create variables dynamically for each trading pair's closing price
for symbol, price in closing_prices.items():
    # Remove special characters to create a valid variable name
    variable_name = symbol.replace('/', '_').replace('-', '_').replace('.', '_')
    globals()[variable_name] = price

# Print the closing prices
for symbol, price in closing_prices.items():
    print(f"{symbol}: {price}")

### Calculating Rates

In [ ]:
# Get the list of columns ending with '_close'
close_columns = [col for col in merged_df.columns if col.endswith('_close')]

# Include the 'date' column along with the 'close' columns
selected_columns = ['date'] + close_columns

# Create a new DataFrame with only the selected columns
df_rates = merged_df[selected_columns]

###### Merging rates with our dataset

In [ ]:
merged_df = df2.merge(df_rates, on='Date', how='left')

In [ ]:
# Define a dictionary mapping 'Buy Cur.' values to their corresponding close column names
cur_to_close = {
    'USDT': None,
    'USDC': None,
    'CRV': 'CRV_USDT_close',
    'XMR': 'XMR_USDT_close',
    'NEO': 'NEO_USDT_close',
    'BTC': 'BTC_USDT_close',
    'BNB': 'BNB_USDT_close',
    'MTL': 'MTL_USDT_close',
    'ZRX': 'ZRX_USDT_close',
    'YFI': 'YFI_USDT_close',
    'ICX': 'ICX_USDT_close',
    'EOS': 'EOS_USDT_close',
    'AION': 'AION_USDT_close',
    'XLM': 'XLM_USDT_close',
    'OMG': 'OMG_USDT_close',
    'XZC': 'XZC_USDT_close',
    'ETH': 'ETH_USDT_close',
    'REN': 'REN_USDT_close',
    'LTC': 'LTC_USDT_close',
    'HOT': 'HOT_USDT_close',
    'MKR': 'MKR_USDT_close',
    'GTO': 'GTO_USDT_close'
}

# Initialize an empty list to store the conditions and results for the np.select function
conditions = []
results = []

# Loop through the 'Buy Cur.' values and construct conditions and results dynamically
for cur, close_column in cur_to_close.items():
    condition = (merged_df['Buy Cur.'] == cur)
    conditions.append(condition)
    
    if close_column is not None:
        result = merged_df['Buy'] * merged_df[close_column]
    else:
        result = 1
    results.append(result)

# Use np.select to create the 'Buy Rate' column based on the conditions and results
merged_df['Buy Rate'] = np.select(conditions, results)

In [ ]:
# Define a dictionary mapping 'Buy Cur.' values to their corresponding close column names
cur_to_close = {
    'USDT': None,
    'USDC': None,
    'CRV': 'CRV_USDT_close',
    'XMR': 'XMR_USDT_close',
    'NEO': 'NEO_USDT_close',
    'BTC': 'BTC_USDT_close',
    'BNB': 'BNB_USDT_close',
    'MTL': 'MTL_USDT_close',
    'ZRX': 'ZRX_USDT_close',
    'YFI': 'YFI_USDT_close',
    'ICX': 'ICX_USDT_close',
    'EOS': 'EOS_USDT_close',
    'AION': 'AION_USDT_close',
    'XLM': 'XLM_USDT_close',
    'OMG': 'OMG_USDT_close',
    'XZC': 'XZC_USDT_close',
    'ETH': 'ETH_USDT_close',
    'REN': 'REN_USDT_close',
    'LTC': 'LTC_USDT_close',
    'HOT': 'HOT_USDT_close',
    'MKR': 'MKR_USDT_close',
    'GTO': 'GTO_USDT_close'
}

# Initialize an empty list to store the conditions and results for the np.select function
conditions = []
results = []

# Loop through the 'Buy Cur.' values and construct conditions and results dynamically
for cur, close_column in cur_to_close.items():
    condition = (merged_df['Sell Cur.'] == cur)
    conditions.append(condition)
    
    if close_column is not None:
        result = merged_df['Sell'] * merged_df[close_column]
    else:
        result = 1
    results.append(result)

# Use np.select to create the 'Buy Rate' column based on the conditions and results
merged_df['Sell Rate'] = np.select(conditions, results)

In [ ]:
# Creating a column for Fair market value of withdrawal
merged_df['FMV Withdrawal'] = pd.NaT

# Set the values in 'FMV Withdrawal' column based on the condition
withdrawal_condition = (merged_df['Type'] == 'Withdrawal') | (merged_df['Type'] == 'Expense (non taxable)')
merged_df.loc[withdrawal_condition, 'FMV Withdrawal'] = merged_df.loc[withdrawal_condition, 'Sell Rate']

In [ ]:
# Creating a column for Fair market value of deposit
merged_df['FMV Deposit'] = pd.NaT

# Set the values in 'FMV Deposit' column based on the condition
deposit_condition = (merged_df['Type'] == 'Deposit') | (merged_df['Type'] == 'Income (non taxable)')
merged_df.loc[deposit_condition, 'FMV Deposit'] = merged_df.loc[deposit_condition, 'Sell Rate']

### Calculating Capital Gains

In [ ]:
#Calculating capital gains on Type
currencies = ['USDT', 'USDC','CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI','ICX', 'EOS', 'AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN','LTC', 'HOT', 'MKR', 'GTO']
capital_gains = []

for currency in currencies:
    # Filter the dataset based on the specified conditions
    sell_df = merged_df[(merged_df['Type'] == 'Trade') & (merged_df['Buy Cur.'] == currency)]
    buy_df = merged_df[(merged_df['Type'] == 'Trade') & (merged_df['Sell Cur.'] == currency)]
    
    # Calculate the sum of 'Sell Rate' and 'Buy Rate' from the filtered datasets
    sell_sum = sell_df['Sell Rate'].sum()
    buy_sum = buy_df['Buy Rate'].sum()
    
    # Calculate the capital gain
    capital_gain = buy_sum - sell_sum
    capital_gains.append(capital_gain)
    
    # Print the capital gain for each currency
    print(f"T_CGT_{currency}:", capital_gain)

# Create a dictionary with the variable names and their corresponding values
data = {
    'Token': [f"T_CGT_{currency}" for currency in currencies],
    'Capital Gain': capital_gains
}

# Create a DataFrame from the dictionary
df_cgt = pd.DataFrame(data)

# Calculate the sum of 'Capital Gain'
T_TCGT = df_cgt['Capital Gain'].sum()

# Add the sum as a new row in the DataFrame
df_cgt.loc['Total'] = ['T_TCGT', T_TCGT]

# Display the DataFrame
print(df_cgt)

# Store the TCGT as a separate variable
T_TCGT_variable = T_TCGT

In [ ]:
#Calculating capital gains on OTHER FEE
# Assuming you have a DataFrame named merged_df
# Define the list of values you want to keep in the "Buy Cur." and "Sell Cur." columns
desired_values = ['USDT', 'USDC','CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI','ICX', 'EOS', 'AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN','LTC', 'HOT', 'MKR', 'GTO']

# Define the conditions for filtering
filter_conditions = (merged_df['Type'].isin(['Other Fees', 'Other Fee']))

# Initialize dictionaries to store cost basis and proceeds for each currency
cost_basis = {}
proceeds = {}

# Calculate cost basis and proceeds for each currency in desired_values
for currency in desired_values:
    filtered_buy_df = merged_df.loc[filter_conditions & (merged_df['Buy Cur.'] == currency)]
    filtered_sell_df = merged_df.loc[filter_conditions & (merged_df['Sell Cur.'] == currency)]

    cost_basis[currency] = filtered_sell_df['Sell Rate'].sum()
    proceeds[currency] = filtered_buy_df['Buy Rate'].sum()

# Calculate total capital gains on Other fees for all currencies
O_TCGT_total = sum(proceeds.values()) - sum(cost_basis.values())

# Print individual cost basis, proceeds, and total capital gains for each currency
for currency in desired_values:
    print(f"O_CB_{currency}:", cost_basis[currency])
    print(f"O_PR_{currency}:", proceeds[currency])

print("O_TCGT_total:", O_TCGT_total)

In [ ]:
#Calculating capital gains on Staking
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for staking
staking_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS','AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the staking_currencies list
staking_currencies = list(set(staking_currencies))

# Initialize dictionaries to store proceeds and capital gains for each staking currency
staking_proceeds = {}
staking_capital_gains = {}
S_TCGT = 0

# Calculate proceeds and capital gains on staking
for currency in staking_currencies:
    staking_df = merged_df[(merged_df['Type'] == 'Staking') & (merged_df['Buy Cur.'] == currency)]
    staking_proceeds[currency] = staking_df['Buy Rate'].sum()
    staking_capital_gains[currency] = staking_proceeds[currency] - staking_df['Sell Rate'].sum()  # Corrected calculation
    print(f"S_PR_{currency}: {staking_proceeds[currency]}")
    print(f"S_CGT_{currency}: {staking_capital_gains[currency]}")
    
    S_TCGT += staking_capital_gains[currency]

print("S_TCGT:", S_TCGT)

In [ ]:
#Calculating capital gains on 'Derivatives / Futures Profit / Futures Loss',
# Calculate sum of "Buy" column for "Derivatives / Futures Profit" type
derivatives_profit_sum = merged_df.loc[merged_df['Type'] == 'Derivatives / Futures Profit', 'Buy'].sum()

# Calculate sum of "Sell" column for "Derivatives / Futures Loss" type
derivatives_loss_sum = merged_df.loc[merged_df['Type'] == 'Derivatives / Futures Loss', 'Sell'].sum()

# Calculate D_TCGT (Derivatives / Futures Total Capital Gains)
D_TCGT = derivatives_profit_sum - derivatives_loss_sum  

# Print the result
print("D_TCGT:", D_TCGT)

In [ ]:
# Calculating capital gains on LP Rewards
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for LP Rewards
lp_rewards_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS','AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the lp_rewards_currencies list
lp_rewards_currencies = list(set(lp_rewards_currencies))

# Initialize dictionaries to store proceeds and capital gains for each LP Rewards currency
lp_rewards_proceeds = {}
lp_rewards_capital_gains = {}
LP_TCGT = 0

# Calculate proceeds and capital gains on LP Rewards
for currency in lp_rewards_currencies:
    lp_rewards_df = merged_df[(merged_df['Type'] == 'LP Rewards') & (merged_df['Buy Cur.'] == currency)]
    lp_rewards_proceeds[currency] = lp_rewards_df['Buy Rate'].sum()
    lp_rewards_capital_gains[currency] = lp_rewards_proceeds[currency] - lp_rewards_df['Sell Rate'].sum()
    print(f"LP_PR_{currency}: {lp_rewards_proceeds[currency]}")
    print(f"LP_CGT_{currency}: {lp_rewards_capital_gains[currency]}")
    
    LP_TCGT += lp_rewards_capital_gains[currency]

print("LP_TCGT:", LP_TCGT)

In [ ]:
# Calculating capital gains on Other Income
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for Other Income
other_income_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS',
                          'AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the other_income_currencies list
other_income_currencies = list(set(other_income_currencies))

# Initialize dictionaries to store proceeds and capital gains for each Other Income currency
other_income_proceeds = {}
other_income_capital_gains = {}
OI_TCGT = 0

# Calculate proceeds and capital gains on Other Income
for currency in other_income_currencies:
    other_income_df = merged_df[(merged_df['Type'] == 'Other Income') & (merged_df['Buy Cur.'] == currency)]
    other_income_proceeds[currency] = other_income_df['Buy Rate'].sum()
    other_income_capital_gains[currency] = other_income_proceeds[currency] - other_income_df['Sell Rate'].sum()
    print(f"OI_PR_{currency}: {other_income_proceeds[currency]}")
    print(f"OI_CGT_{currency}: {other_income_capital_gains[currency]}")
    
    OI_TCGT += other_income_capital_gains[currency]

print("OI_TCGT:", OI_TCGT)

In [ ]:
# Calculating capital gains on Other Expense
# Calculating capital losses on Other Expense
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for Other Expense
other_expense_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS','AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the other_expense_currencies list
other_expense_currencies = list(set(other_expense_currencies))

# Initialize the variable to store the total capital losses for Other Expense
OE_TCGT = 0

# Calculate capital losses on Other Expense
for currency in other_expense_currencies:
    other_expense_df = merged_df[(merged_df['Type'] == 'Other Expense') & (merged_df['Sell Cur.'] == currency)]
    capital_losses = -(other_expense_df['Sell Rate'].sum())  # Multiply with -1 to account for losses
    print(f"OE_CGT_{currency}: {capital_losses}")
    
    OE_TCGT += capital_losses

print("OE_TCGT:", OE_TCGT)

In [ ]:
# Calculating capital losses on Donation
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for Donation
donation_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS','AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the donation_currencies list
donation_currencies = list(set(donation_currencies))

# Initialize the variable to store the total capital losses for Donation
DN_TCGT = 0

# Calculate capital losses on Donation
for currency in donation_currencies:
    donation_df = merged_df[(merged_df['Type'] == 'Donation') & (merged_df['Sell Cur.'] == currency)]
    capital_losses = -(donation_df['Sell Rate'].sum())  # Multiply with -1 to account for losses
    print(f"DN_CGT_{currency}: {capital_losses}")
    
    DN_TCGT += capital_losses

print("DN_TCGT:", DN_TCGT)

In [ ]:
# Calculating capital losses on Spend
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for Spend
spend_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS','AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the spend_currencies list
spend_currencies = list(set(spend_currencies))

# Initialize the variable to store the total capital losses for Spend
SP_TCGT = 0

# Calculate capital losses on Spend
for currency in spend_currencies:
    spend_df = merged_df[(merged_df['Type'] == 'Spend') & (merged_df['Sell Cur.'] == currency)]
    capital_losses = -(spend_df['Sell Rate'].sum())  # Multiply with -1 to account for losses
    print(f"SP_CGT_{currency}: {capital_losses}")
    
    SP_TCGT += capital_losses

print("SP_TCGT:", SP_TCGT)

In [ ]:
# Calculating capital gains on Airdrop
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for Airdrop
airdrop_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS','AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the airdrop_currencies list
airdrop_currencies = list(set(airdrop_currencies))

# Initialize dictionaries to store proceeds and capital gains for each airdrop currency
airdrop_proceeds = {}
airdrop_capital_gains = {}
AI_TCGT = 0

# Calculate proceeds and capital gains on Airdrop
for currency in airdrop_currencies:
    airdrop_df = merged_df[(merged_df['Type'] == 'Airdrop') & (merged_df['Buy Cur.'] == currency)]
    airdrop_proceeds[currency] = airdrop_df['Buy Rate'].sum()
    airdrop_capital_gains[currency] = airdrop_proceeds[currency] - airdrop_df['Sell Rate'].sum()
    print(f"AI_PR_{currency}: {airdrop_proceeds[currency]}")
    print(f"AI_CGT_{currency}: {airdrop_capital_gains[currency]}")
    
    AI_TCGT += airdrop_capital_gains[currency]

print("AI_TCGT:", AI_TCGT)

In [ ]:
# Calculating capital gains on Reward / Bonus
# Assuming you have a DataFrame named merged_df

# Define the list of currencies for Reward / Bonus
reward_bonus_currencies = ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS',
                           'AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']

# Remove duplicates (if any) from the reward_bonus_currencies list
reward_bonus_currencies = list(set(reward_bonus_currencies))

# Initialize dictionaries to store proceeds and capital gains for each Reward / Bonus currency
reward_bonus_proceeds = {}
reward_bonus_capital_gains = {}
RB_TCGT = 0

# Calculate proceeds and capital gains on Reward / Bonus
for currency in reward_bonus_currencies:
    reward_bonus_df = merged_df[(merged_df['Type'] == 'Reward / Bonus') & (merged_df['Buy Cur.'] == currency)]
    reward_bonus_proceeds[currency] = reward_bonus_df['Buy Rate'].sum()
    reward_bonus_capital_gains[currency] = reward_bonus_proceeds[currency] - reward_bonus_df['Sell Rate'].sum()
    print(f"RB_PR_{currency}: {reward_bonus_proceeds[currency]}")
    print(f"RB_CGT_{currency}: {reward_bonus_capital_gains[currency]}")
    
    RB_TCGT += reward_bonus_capital_gains[currency]

print("RB_TCGT:", RB_TCGT)

In [ ]:
#Total capital gains
TCGT = T_TCGT + O_TCGT_total + S_TCGT + D_TCGT + LP_TCGT + OI_TCGT + OE_TCGT + DN_TCGT + SP_TCGT + AI_TCGT + RB_TCGT
TCGT

In [ ]:
#Tax on Capital gains (assuming tax is 30%)
PER_TCGT = 0.3 * TCGT
PER_TCGT

# Monitoring

### Volume Based Monitoring

In [ ]:
# Initialize a dictionary to store results
results = {}

for currency in ['USDT', 'USDC', 'CRV', 'XMR', 'NEO', 'BTC', 'BNB', 'MTL', 'ZRX', 'YFI', 'ICX', 'EOS', 'AION', 'XLM', 'OMG', 'XZC', 'ETH', 'REN', 'LTC', 'HOT', 'MKR', 'GTO']:
    # Step 1: Sum all values in "Sell" column where "Sell Cur." is the current currency
    sell_sum = merged_df.loc[merged_df['Sell Cur.'] == currency, 'Sell'].sum()
    
    # Step 2: Sum all values in "Buy" column where "Buy Cur." is the current currency
    buy_sum = merged_df.loc[merged_df['Buy Cur.'] == currency, 'Buy'].sum()
    
    # Step 3: Calculate the difference between buy_sum and sell_sum
    diff = sell_sum - buy_sum  
    
    # Step 4: Multiply the difference with the appropriate value
    if currency in ['USDT', 'USDC']:
        result = diff * 1  # Multiply by 1 for USDT and USDC
    elif currency + '/USDT' in closing_prices:
        result = diff * closing_prices[currency + '/USDT']
        results[currency] = result

# Calculate the total closing balance
closing_balance = sum(results.values())

# Print the individual results
for currency, result in results.items():
    print(f"{currency}: {result}")

# Print the total closing balance
print(f"Closing Balance: {closing_balance}")

### Risk Based Monitoring

In [ ]:
#MAKING DATA BASE
import sqlite3
conn = sqlite3.connect('RISKY_ACCOUNTS.db')

In [ ]:
# Replace 'file1.csv' with the path to your first CSV file
Risky_CEX = pd.read_csv('Risky_CEX.csv')
P2P = pd.read_csv('P2P.csv')
Dark_Service = pd.read_csv('Dark_Service.csv')
Sanction = pd.read_csv('Sanction.csv')
Stolen_Coins = pd.read_csv('Stolen_Coins.csv')
Gambling = pd.read_csv('Gambling.csv')
Enforcement = pd.read_csv('Enforcement.csv')
Scam = pd.read_csv('Scam.csv')
Prepaid_Cryptocurrency_Cards = pd.read_csv('Prepaid_Cryptocurrency_Cards.csv')
BTC_ATM = pd.read_csv('BTC_ATM.csv')

# Create a SQLite database connection
conn = sqlite3.connect('my_database.db')  # Replace 'my_database.db' with your desired database name

# Replace 'table_name1' with the desired name for the first table in the database
Risky_CEX.to_sql('Risky_CEX', conn, index=False, if_exists='replace')
# Repeat the process for other CSV files and tables
P2P.to_sql('P2P', conn, index=False, if_exists='replace')
Dark_Service.to_sql('Dark_Service', conn, index=False, if_exists='replace')
Sanction.to_sql('Sanction', conn, index=False, if_exists='replace')
Stolen_Coins.to_sql('Stolen_Coins', conn, index=False, if_exists='replace')
Gambling.to_sql('Gambling', conn, index=False, if_exists='replace')
Enforcement.to_sql('Enforcement', conn, index=False, if_exists='replace')
Scam.to_sql('Scam', conn, index=False, if_exists='replace')
Prepaid_Cryptocurrency_Cards.to_sql('Prepaid_Cryptocurrency_Cards', conn, index=False, if_exists='replace')
BTC_ATM.to_sql('BTC_ATM', conn, index=False, if_exists='replace')

# Close the database connection
conn.close()

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')  # Replace 'my_database.db' with your actual database name
# Step 3: Loop through all tables in the database and compare with 'merged_df'
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
conn.close()

In [ ]:
#CHECKING IT AGAINST OUR DATASET
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')  # Replace 'my_database.db' with your actual database name
# Step 3: Loop through all tables in the database and compare with 'merged_df'
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()


for table in tables:
    table_name = table[0]
    print(f"\nTable Name: {table_name}")
    # Read the table from the database
    df_table = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    print("Table Content:")
    print(df_table.head())
    # Check for matching values in 'merged_df' and print the table name if there is a match
    matching_values = df_table['Accounts'].isin(df['From'])
    print("Matching Values:")
    print(matching_values)
    if matching_values.any():
        print(f"Matching values found in table: {table_name}")

# Step 4: Close the database connection
conn.close()

### Behavior based Monitoring:

In [ ]:
#FOR MONTHLY DEPOSIT/WITHDRAWAL
# Extract both month and year from the 'Date' column and create a new 'Month' column
merged_df['Month'] = merged_df['Date'].dt.to_period('M').astype(str)

# Group the data by month and year and calculate the sum of deposits and withdrawals for each month
monthly_data = merged_df.groupby(['Month']).agg({
    'FMV Withdrawal': 'sum',
    'FMV Deposit': 'sum'
}).reset_index()

# Calculate the difference (Deposit - Withdrawal)
monthly_data['Deposit - Withdrawal'] = monthly_data['FMV Deposit'] - monthly_data['FMV Withdrawal']

# Print the result
print(monthly_data)

In [ ]:
# Filter the DataFrame for transactions with 'Buy Cur.' or 'Sell Cur.' as 'USDT' or 'USDC'
stablecoin_df = merged_df[(merged_df['Buy Cur.'] == 'USDT') | (merged_df['Buy Cur.'] == 'USDC') |
                          (merged_df['Sell Cur.'] == 'USDT') | (merged_df['Sell Cur.'] == 'USDC')]

# Calculate the total stablecoin transactions for the entire dataset
total_stablecoin_transactions = stablecoin_df.shape[0]

# Calculate the total transactions for the entire dataset
total_transactions = merged_df.shape[0]

# Calculate the overall percentage of stablecoin transactions
overall_percentage = (total_stablecoin_transactions / total_transactions) * 100

# Group the filtered data by month and year to get the total stablecoin transactions for each month
stablecoin_transactions = stablecoin_df.groupby(['Month']).size().reset_index(name='Stablecoin Transactions')

# Group the original 'merged_df' DataFrame by month and year to get the total transactions for each month
total_transactions_monthly = merged_df.groupby(['Month']).size().reset_index(name='Total Transactions')

# Merge the two DataFrames on 'Month' to calculate the percentage
result_df = pd.merge(total_transactions_monthly, stablecoin_transactions, on='Month', how='left')

# Calculate the monthly percentage of stablecoin transactions
result_df['Monthly Percentage'] = (result_df['Stablecoin Transactions'] / result_df['Total Transactions']) * 100

# Calculate the total monthly percentage compared to the overall percentage
result_df['Total Monthly Percentage'] = (result_df['Monthly Percentage'] / overall_percentage) * 100

# Calculate the overall monthly percentage for the entire dataset
overall_monthly_percentage = (result_df['Stablecoin Transactions'].sum() / result_df['Total Transactions'].sum()) * 100

# Print the result
result_df

In [ ]:
#Movement Of Stablecoins Across Multiple Wallets.
# Filter the DataFrame for transactions with 'Buy Cur.' or 'Sell Cur.' as 'USDT' or 'USDC'
filtered_df = merged_df[(merged_df['Buy Cur.'] == 'USDT') | (merged_df['Buy Cur.'] == 'USDC') |
                        (merged_df['Sell Cur.'] == 'USDT') | (merged_df['Sell Cur.'] == 'USDC')]

# Group the filtered data by month and year and calculate the sum of deposits and withdrawals for each month
monthly_data = filtered_df.groupby(['Month']).agg({
    'FMV Withdrawal': 'sum',
    'FMV Deposit': 'sum'
}).reset_index()

# Calculate the difference (Deposit - Withdrawal)
monthly_data['Deposit - Withdrawal'] = monthly_data['FMV Deposit'] - monthly_data['FMV Withdrawal']

# Print the result
print(monthly_data)

In [ ]:
# Calculate the monthly closing balance
monthly_closing_balance = merged_df.groupby('Month').apply(lambda x: (x['Sell Rate'] - x['Buy Rate']).sum()).reset_index(name='Closing Balance')

# Print the result
print(monthly_closing_balance)

In [ ]:
# Calculate the yearly closing balance
yearly_closing_balance = merged_df.groupby(merged_df['Date'].dt.year)['Sell Rate', 'Buy Rate'].sum()
yearly_closing_balance['Closing Balance'] = yearly_closing_balance['Sell Rate'] - yearly_closing_balance['Buy Rate']

# Print the result
print(yearly_closing_balance)

In [ ]:
#Decreasing Volume Over The Time:

# Initialize a variable to keep track of the consecutive occurrences
consecutive_occurrences = 0

# Loop through the 'Closing Balance' column (start from index 1)
for i in range(1, len(monthly_closing_balance)):
    current_balance = monthly_closing_balance.iloc[i]['Closing Balance']
    previous_balance = monthly_closing_balance.iloc[i - 1]['Closing Balance']

    # Check if the current balance is 20% less than the previous balance
    if current_balance < 0.8 * previous_balance:
        consecutive_occurrences += 1
    else:
        # If the current balance is not 20% less than the previous, reset the consecutive_occurrences
        consecutive_occurrences = 0

    # Check if consecutive occurrences have reached 3, then break the loop (no need to check further)
    if consecutive_occurrences == 3:
        break

# Determine the corresponding risk percentage
if consecutive_occurrences == 1:
    risk_percentage = 5
elif consecutive_occurrences == 2:
    risk_percentage = 10
elif consecutive_occurrences >= 3:
    risk_percentage = 20
else:
    risk_percentage = 0  # No consecutive occurrences

# Print the result
print("Risk Percentage:", risk_percentage, "%")

# Fraudulent transactions detection using ML

In [ ]:
merged_df.to_csv('Data_ML.csv', index=False) 

In [ ]:
Data_ML = pd.read_csv('F:\Data_ML.csv')

In [ ]:
Data_ML

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Encode categorical features using one-hot encoding
categorical_features = ['Type', 'Buy Cur.', 'Sell Cur.', 'Exchange']
Data_ML_encoded = pd.get_dummies(Data_ML, columns=categorical_features)

# Define features and target variable
X = Data_ML_encoded.drop(columns=['Fraudulent'])
y = Data_ML_encoded['Fraudulent']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Gaussian Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = nb_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

In [ ]:
print('20%')

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Assuming you have already encoded categorical variables and prepared your data
# Selecting features
feature_columns = ['Buy', 'Sell']
X = Data_ML[feature_columns]

# Selecting target variable
y = Data_ML['Fraudulent']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Initialize and train the Logistic Regression model
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train_imputed, y_train)

# Predict on the test set
y_pred = logreg_model.predict(X_test_imputed)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)